1 of 6: install correct Python version; setup virtual environment; install pip


In [1]:
!apt-get update -y
!apt-get install python3.10 python3.10-distutils python3.10-venv -y


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 384 kB in 2s (174 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [2]:
!python3.10 -m venv /content/dmo_env


In [3]:
%%bash
source /content/dmo_env/bin/activate

# Explicitly install pip within your existing venv
python -m ensurepip --upgrade

# Explicitly verify pip installation
which pip
pip --version


Looking in links: /tmp/tmpmyfldebt
/content/dmo_env/bin/pip
pip 22.0.2 from /content/dmo_env/lib/python3.10/site-packages/pip (python 3.10)


2 of 6: ensure GPU is available (e.g., runtime= "Connect T4") before cloning/downloading; show initial state of GPU

In [4]:
# GPU check explicitly
import torch

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU is available: {device_name}")
else:
    print("❌ GPU is not available. Ensure GPU runtime is enabled.")


✅ GPU is available: Tesla T4


In [5]:
!nvidia-smi

Fri Jul 25 19:45:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

3 of 6: clone (fork of) DMOSpeech2

In [6]:
import os

# Explicitly move to a known good directory
os.chdir('/content')

# Verify explicitly you're back in a good state
print("Current working directory explicitly set to:", os.getcwd())

!git clone https://github.com/wrightmikea/DMOSpeech2.git
%cd DMOSpeech2
!echo "pwd=`pwd`"

Current working directory explicitly set to: /content
fatal: destination path 'DMOSpeech2' already exists and is not an empty directory.
/content/DMOSpeech2
pwd=/content/DMOSpeech2


4 of 6: install requirements into activated virtual environment

In [7]:

!source /content/dmo_env/bin/activate && pip install -r requirements.txt


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121


5 of 6: fetch the DMOSpeech2 PyTorch model checkpoints from Huggingface into the cloned (forked) repo

In [8]:
import os
os.chdir('/content/DMOSpeech2')

# Explicitly download checkpoints
!mkdir -p ckpts
!wget -O ckpts/model_85000.pt https://huggingface.co/yl4579/DMOSpeech2/resolve/main/model_85000.pt
!wget -O ckpts/model_1500.pt https://huggingface.co/yl4579/DMOSpeech2/resolve/main/model_1500.pt


--2025-07-25 19:45:27--  https://huggingface.co/yl4579/DMOSpeech2/resolve/main/model_85000.pt
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.114, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/cd/e6/cde65999e1daa02ab5a64eef05158e83369c7b7b1bf9eab45cd2b2aa4f644396/e143188f7c46b69abb60786937d79b89dac9777d3c4c1e91a97634bcd39ebf2c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model_85000.pt%3B+filename%3D%22model_85000.pt%22%3B&Expires=1753476334&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MzQ3NjMzNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2NkL2U2L2NkZTY1OTk5ZTFkYWEwMmFiNWE2NGVlZjA1MTU4ZTgzMzY5YzdiN2IxYmY5ZWFiNDVjZDJiMmFhNGY2NDQzOTYvZTE0MzE4OGY3YzQ2YjY5YWJiNjA3ODY5MzdkNzliODlkYWM5Nzc3ZDNjNGMxZTkxYTk3NjM0YmNkMzllYmYyYz9yZXNwb25zZS1jb250ZW50LWRpc3

6 of 6: verify environment; run the app to show the Gradio UI

In [9]:
import subprocess

venv_path = "/content/dmo_env"
python_bin = f"{venv_path}/bin/python"

def run_command(cmd):
    result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE, text=True)
    return result.stdout.strip() if result.stdout else result.stderr.strip()

print("🔍 Explicit verification after corrected installation:")

# Verify PyTorch explicitly
torch_version = run_command(f'{python_bin} -c "import torch; print(torch.__version__)"')
print(f"• PyTorch version explicitly: {torch_version}")

# Verify torchaudio explicitly
torchaudio_version = run_command(f'{python_bin} -c "import torchaudio; print(torchaudio.__version__)"')
print(f"• torchaudio version explicitly: {torchaudio_version}")

# Verify CUDA explicitly
cuda_available = run_command(f'{python_bin} -c "import torch; print(torch.cuda.is_available())"')
print(f"• CUDA explicitly available: {cuda_available}")

# Explicitly confirm accelerate package explicitly installed
accelerate_version = run_command(f'{python_bin} -c "import accelerate; print(accelerate.__version__)"')
print(f"• accelerate explicitly installed: {accelerate_version}")


🔍 Explicit verification after corrected installation:
• PyTorch version explicitly: 2.2.2+cu121
• torchaudio version explicitly: 2.2.2+cu121
• CUDA explicitly available: True
• accelerate explicitly installed: 1.9.0


In [ ]:
!source /content/dmo_env/bin/activate && python --version && python scripts/colab-gradio.py

Python 3.10.12
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.656 seconds.
Prefix dict has been built successfully.
Word segmentation module jieba initialized.

Download Vocos from huggingface charactr/vocos-mel-24khz
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://59ae32e04bb6f0614a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Converting audio...
Using custom reference text...

ref_text   Is ant it ironic?. 
/content/dmo_env/lib/python3.10/site-packages/gradio/processing_utils.py:777: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
